In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 6.4 MB/s eta 0:00:00


In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken

In [4]:
import os

# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


Please enter your OpenAI API key: sk-s0ugDUzhON6qwMa_a4Qi6k2TIrZcIA9dXrJvDQ8d39T3BlbkFJfYPGdAJSlINUTuaISEsBmaeoPDfF5OVqBOkJfFV2IA
OPENAI_API_KEY has been set!


In [5]:
llm_model = "gpt-4o-mini"

In [6]:
from langchain.text_splitter import CharacterTextSplitter

In [7]:
pdf_file_path = './PresentacioCurs24-25.pdf'
loader = PyPDFLoader(file_path=pdf_file_path)
data = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = text_splitter.split_documents(data)

In [8]:
data

[Document(metadata={'source': '/content/drive/MyDrive/PresentacioCurs24-25.pdf', 'page': 0}, page_content='PRESENTACIÓ DEL CURS\nPROGRAMACIÓ I - GRAU EI - CURS 24/25\nINMACULADA RODRIGUEZ SANTIAGO'),
 Document(metadata={'source': '/content/drive/MyDrive/PresentacioCurs24-25.pdf', 'page': 1}, page_content='2\nCONTINGUT\n▪ PROFESSORAT\n▪ TEMARI\n▪ METODOLOGIA, HORARIS, PLANIFICACIÓ\n▪ AVALUACIÓ\n▪ CONSIDERACIONS ÈTIQUES\n▪ BIBLIOGRAFIA'),
 Document(metadata={'source': '/content/drive/MyDrive/PresentacioCurs24-25.pdf', 'page': 2}, page_content='PROFESSORAT\n▪ Inmaculada Rodriguez Santiago (Teoria, Problemes) \ninmarodriguez@ub.edu\n▪ Albert Clapés (Pràctiques) aclapes@ub.edu\n▪ Julio Jacques Silveira (Pràctiques) julio.silveira@ub.edu\n▪ Pau Soler (Pràctiques) pau.soler@ub.edu\n3'),
 Document(metadata={'source': '/content/drive/MyDrive/PresentacioCurs24-25.pdf', 'page': 3}, page_content='TEMARI 4\n1. Programació bàsica: variables i constants, tipus de dades, expressions i \nsentències ele

In [9]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(data, embedding=embeddings)

<ipython-input-9-1ad7d04367c8>:2: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [10]:
# Create conversation chain
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")
memory = ConversationBufferMemory(
memory_key='chat_history', return_messages=True)
conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(),
        memory=memory
        )

<ipython-input-10-adaa16b5973e>:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")
<ipython-input-10-adaa16b5973e>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [11]:
query = "Qui son els professors"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

<ipython-input-11-f5485dd7c1c9>:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query})


'Els professors són:\n\n- Inmaculada Rodriguez Santiago (Teoria, Problemes) - inmarodriguez@ub.edu\n- Albert Clapés (Pràctiques) - aclapes@ub.edu\n- Julio Jacques Silveira (Pràctiques) - julio.silveira@ub.edu\n- Pau Soler (Pràctiques) - pau.soler@ub.edu'

In [12]:
query = "What is the contact information?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

'La informació de contacte dels professors és la següent:\n\n- Inmaculada Rodriguez Santiago (Teoria, Problemes): inmarodriguez@ub.edu\n- Albert Clapés (Pràctiques): aclapes@ub.edu\n- Julio Jacques Silveira (Pràctiques): julio.silveira@ub.edu\n- Pau Soler (Pràctiques): pau.soler@ub.edu'

In [13]:
query = "Quan és l'examen parcial?"
result = conversation_chain({"question": query})
answer = result["answer"]
answer

"L'examen parcial 1 és el dijous 7 de novembre de 2024, de 15:00 a 18:00. L'examen parcial 2 és el divendres 10 de gener de 2025, de 9:00 a 14:00."